# 📚 Kapitola 22: Docker Compose

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 3 | ADVANCED</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **docker-compose.yml structure**
- **Services definition**
- **Multi-container applications**
- **Networks a volumes in compose**
- **Environment variables**
- **depends_on a healthchecks**
- **Override files**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 21

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 22

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 22: Docker Compose")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# Docker Compose – Detailní teoretický výklad

## 1. Úvod a motivace

Docker Compose je nástroj, který umožňuje definovat a spouštět vícekontejnerové aplikace pomocí jednoho jediného konfiguračního souboru – `docker-compose.yml`. V praxi se používá všude, kde je potřeba spravovat složitější systémy, jako jsou webové aplikace s databází, backendem a frontendem, nebo mikroslužby. Pokud pracuješ s Dockerem, pak Docker Compose je nezbytný nástroj pro efektivní práci ve vývoji, testování i produkčním prostředí.

Student se naučí, jak zefektivnit vývoj složitějších aplikací pomocí jednoduchého konfiguračního souboru. Naučí se definovat služby, spravovat sítě a svazky, nastavovat proměnné prostředí a zajišťovat správné spouštění kontejnerů v určitém pořadí.

---

## 2. Hlavní koncepty

### Docker Compose File Structure (Struktura souboru)

Soubor `docker-compose.yml` je YAML soubor, který definuje konfiguraci jedné nebo více služeb (kontejnerů), které spolu komunikují. Tato struktura je jednoduchá, ale velmi výkonná.

```yaml
version: '3.8'
services:
  web:
    image: nginx:latest
    ports:
      - "80:80"
  db:
    image: postgres:13
    environment:
      POSTGRES_PASSWORD: password
```

Tento jednoduchý příklad definoval dvě služby – `web` (nginx) a `db` (postgres), které spolu komunikují. Každá služba je definována v sekci `services`, kde můžeš nastavit image, porty, proměnné prostředí apod.

---

### Services Definition (Definice služeb)

Každá služba v Docker Compose představuje kontejner, který se spouští. Můžeš definovat jejich image, porty, proměnné prostředí, svazky a mnoho dalších vlastností.

```yaml
services:
  app:
    image: node:16
    working_dir: /app
    command: npm start
    volumes:
      - .:/app
    ports:
      - "3000:3000"
```

V tomto příkladu se spouští kontejner s Node.js, který běží na portu 3000 a svazuje aktuální adresář s `/app` uvnitř kontejneru.

---

### Multi-container Applications (Aplikace s více kontejnery)

Docker Compose je ideální nástroj pro vývoj aplikací, které se skládají z více komponent – např. frontend, backend a databáze. Každá komponenta běží ve svém kontejneru, ale komunikují mezi sebou prostřednictvím Docker sítí.

```yaml
services:
  frontend:
    image: nginx
    ports:
      - "80:80"
  backend:
    image: node:16
    depends_on:
      - db
  db:
    image: postgres:13
```

Zde se spouští tři služby: frontend (nginx), backend (Node.js) a databáze (PostgreSQL). Backend závisí na databázi, takže se spustí po ní.

---

### Networks a Volumes (Sítě a svazky)

Docker Compose automaticky vytváří sítě mezi kontejnery. Výchozí síť je pojmenovaná podle jména adresáře, ve kterém se nachází `docker-compose.yml`.

```yaml
networks:
  my-network:
    driver: bridge

services:
  app:
    image: nginx
    networks:
      - my-network
```

Svazky se používají pro trvalé ukládání dat. Můžeš použít named svazky nebo bind mounty.

```yaml
volumes:
  db-data:

services:
  db:
    image: postgres:13
    volumes:
      - db-data:/var/lib/postgresql/data
```

---

### Environment Variables (Proměnné prostředí)

Základním způsobem správy proměnných prostředí je použití sekce `environment`:

```yaml
services:
  app:
    image: node:16
    environment:
      NODE_ENV: production
      DB_HOST: db
```

Můžeš také použít soubor s proměnnými prostředí (`env_file`):

```yaml
services:
  app:
    image: node:16
    env_file:
      - .env
```

---

### depends_on a Healthchecks (Závislosti a kontrola stavu)

Klíčové slovo `depends_on` říká, že daná služba se spustí až po dokončení jiných služeb. Všimni si, že tento mechanismus **nečeká na spuštění aplikace**, ale pouze na spuštění kontejneru.

```yaml
services:
  backend:
    image: node:16
    depends_on:
      db:
        condition: service_healthy
  db:
    image: postgres:13
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U postgres"]
      interval: 10s
      timeout: 5s
      retries: 5
```

Zde se backend spustí až po zajištění, že `db` je zdravý (tj. připravená k přijímání spojení). `healthcheck` je vlastnost, která umožňuje Dockeru kontrolovat stav služby.

---

### Override Files (Přepsací soubory)

Docker Compose podporuje přepsací soubory, které umožňují upravovat konfiguraci pro různá prostředí – vývoj, test, produkce.

```bash
docker-compose -f docker-compose.yml -f docker-compose.override.yml up
```

Použití `override` souborů umožňuje mít jeden hlavní konfigurační soubor a jednotlivé přepisy pro různá prostředí. Například:

```yaml
# docker-compose.override.yml
services:
  app:
    ports:
      - "3000:3000"
```

---

## 3. Důležité detaily

Při práci s Docker Compose se často setkáváme s chybami, jako je špatná konfigurace sítě, nesprávné použití `depends_on`, nebo zanedbané proměnné prostředí. Nejčastější problém je, že se služba spustí, ale nemůže komunikovat kvůli špatně nastavené síti nebo závislosti.

**Best practices:**
- Vždy používej `version` v `docker-compose.yml`
- Používej `healthcheck`, aby systém věděl, kdy je služba připravená
- Nastav `depends_on` spolu s `condition: service_healthy` pro zajištění správného pořadí
- Používej named volumes pro trvalé datové ukládání

**Performance tipy:**
- Vyhněte se používání `build:` pokud není nutné (použijte předem vytvořené image)
- Využij `--no-deps` při spouštění konkrétních služeb
- Používej `docker-compose down` k čistému zastavení a odstranění kontejnerů

---

## 4. Propojení s předchozími kapitolami

Toto téma navazuje na základní znalosti Dockeru – image, kontejnery, sítě a svazky. Docker Compose je rozšířením těchto poznatků a umožňuje efektivní správu vícekontejnerových aplikací.

Pokud jsi již většinu z těchto konceptů ovládal, Docker Compose ti umožní automatizovat a zjednodušit tvůj pracovní postup. Získáš schopnost definovat celý systém jednoduše v jednom souboru a spouštět jej pomocí jednoho příkazu – tedy vývoj se stane rychlejší, efektivnější a snazší.

---


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


# 1. Základní struktura docker-compose.yml

## Co demonstruje:
Tento příklad ukazuje základní strukturu souboru `docker-compose.yml` a definici jednoho služby.

```yaml
version: '3.8'
services:
  web:
    image: nginx:alpine
    ports:
      - "8080:80"
    volumes:
      - ./html:/usr/share/nginx/html

# Spuštění příkazem: docker-compose up
```

## Očekávaný výstup:
- Webový server NGINX běží na portu 8080
- Výstup se zobrazí v konzoli
- Můžete přistoupit na http://localhost:8080

## Vysvětlení:
- `version` - verze Docker Compose formátu
- `services` - definice kontejnerů
- `web` - název služby
- `image` - použitý obraz
- `ports` - mapování portů (host:container)
- `volumes` - připojení lokálního adresáře do kontejneru

---

# 2. Vícekontejnerová aplikace s databází

## Co demonstruje:
Vytvoření dvou kontejnerů - webové aplikace a databáze, které komunikují mezi sebou.

```yaml
version: '3.8'
services:
  web:
    image: node:alpine
    ports:
      - "3000:3000"
    volumes:
      - .:/app
    working_dir: /app
    command: npm start
    depends_on:
      - db
    environment:
      DB_HOST: db
      DB_PORT: 5432

  db:
    image: postgres:13-alpine
    environment:
      POSTGRES_DB: myapp
      POSTGRES_USER: user
      POSTGRES_PASSWORD: password
    volumes:
      - db_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

volumes:
  db_data:

# Spuštění příkazem: docker-compose up
```

## Očekávaný výstup:
- Databáze PostgreSQL běží na portu 5432
- Webová aplikace Node.js běží na portu 3000
- Služby se spouští v pořadí (nejdříve DB, pak web)

## Vysvětlení:
- `depends_on` - zajišťuje pořadí spuštění kontejnerů
- `environment` - nastavení proměnných prostředí
- `volumes` - trvalé úložiště pro databázi
- `working_dir` a `command` - nastavení pracovního adresáře a spuštění aplikace

---

# 3. Síťování a proměnné prostředí

## Co demonstruje:
Použití vlastní sítě, proměnných prostředí a sdílení dat mezi kontejnery.

```yaml
version: '3.8'
services:
  app1:
    image: alpine:latest
    command: ping -c 5 ${TARGET_HOST}
    environment:
      TARGET_HOST: app2
    networks:
      - mynetwork

  app2:
    image: alpine:latest
    command: httpd -f -p 8080
    ports:
      - "8080:8080"
    networks:
      - mynetwork

networks:
  mynetwork:
    driver: bridge

# Spuštění příkazem: docker-compose up
```

## Očekávaný výstup:
- Kontejner app1 pingne hostitele app2
- Kontejner app2 spustí HTTP server na portu 8080
- Všechny kontejnery jsou v jedné vlastní síti

## Vysvětlení:
- `networks` - vlastní síť pro izolaci kontejnerů
- `${TARGET_HOST}` - proměnná prostředí zadaná v souboru
- `command` - příkaz spouštěný v kontejneru

---

# 4. Healthcheck a závislosti

## Co demonstruje:
Zavedení kontrolních mechanismů pro sledování stavu kontejnerů a jejich závislostí.

```yaml
version: '3.8'
services:
  backend:
    image: node:alpine
    ports:
      - "3000:3000"
    command: npm start
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:3000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
    depends_on:
      db:
        condition: service_healthy

  db:
    image: postgres:13-alpine
    environment:
      POSTGRES_DB: testdb
      POSTGRES_USER: user
      POSTGRES_PASSWORD: password
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U user -d testdb"]
      interval: 10s
      timeout: 5s
      retries: 5

# Spuštění příkazem: docker-compose up
```

## Očekávaný výstup:
- Kontejner DB se spustí a zkontroluje svůj stav
- Backend čeká na zdravý stav databáze
- Výstup ukazuje, že kontejnery jsou spuštěné a zdravé

## Vysvětlení:
- `healthcheck` - kontrola stavu kontejneru
- `condition: service_healthy` - čekání na zdravý stav závislého kontejneru
- `test` - příkaz pro testování stavu služby

---

# 5. Přepsání konfigurace pomocí override souboru

## Co demonstruje:
Použití `docker-compose.override.yml` pro přepsání nastavení v hlavním souboru.

**docker-compose.yml**
```yaml
version: '3.8'
services:
  web:
    image: nginx:alpine
    ports:
      - "8080:80"
    volumes:
      - ./html:/usr/share/nginx/html
    environment:
      ENV: production

# Spuštění příkazem: docker-compose up
```

**docker-compose.override.yml**
```yaml
version: '3.8'
services:
  web:
    ports:
      - "8081:80"
    environment:
      ENV: development
    volumes:
      - ./html:/usr/share/nginx/html
      - ./logs:/var/log/nginx

# Spuštění příkazem: docker-compose up
```

## Očekávaný výstup:
- Webový server běží na portu 8081 (přepsané)
- V prostředí je nastaveno `ENV=development`
- Přidán přístup k log souborům

## Vysvětlení:
- `docker-compose.override.yml` se automaticky načte, pokud existuje
- Nastavení v override přepisují nastavení v hlavním souboru
- Může být použito pro různá prostředí (dev, test, prod)

---

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# 5 praktických cvičení pro Docker Compose

---

## Cvičení 1: **Vytvoření multi-containr aplikace s PostgreSQL a Node.js**

### 1. **Název úkolu**
Multi-containr aplikace: Node.js backend + PostgreSQL databáze

### 2. **Detailní zadání**
Vytvoř Docker Compose konfiguraci, která spustí dvě služby:
- Node.js aplikaci (např. Express server) s jednoduchým API pro práci s uživateli
- PostgreSQL databázi s inicializací tabulky `users` a několika testovacími daty

Aplikace by měla být dostupná na portu 3000 a připojovat se k DB.

### 3. **Vstupní data/požadavky**
- Node.js image: `node:18-alpine`
- PostgreSQL image: `postgres:15-alpine`
- Databáze má být pojmenovaná `myapp_db`
- Uživatelské jméno a heslo pro DB: `user` / `password`
- Aplikace musí běžet na portu 3000

### 4. **Očekávaný výstup**
- Docker Compose soubor (`docker-compose.yml`) se dvěma službami
- Spuštěná aplikace na `localhost:3000`
- Funkční API endpoint `/users` pro získání seznamu uživatelů
- V databázi jsou vložena testovací data

### 5. **Hints/Nápověda**
1. Použij `depends_on` pro řízení pořadí spuštění služeb
2. Nastav proměnné prostředí pro připojení k DB v Node.js aplikaci
3. Využij `volumes` pro přidání init skriptu nebo dat do PostgreSQL
4. Ujisti se, že porty jsou správně mapovány

### 6. **Kostra řešení**
```yaml
version: '3.8'
services:
  app:
    image: node:18-alpine
    ports:
      - "3000:3000"
    depends_on:
      - db
    environment:
      - DB_HOST=db
      - DB_USER=user
      - DB_PASSWORD=password
      - DB_NAME=myapp_db

  db:
    image: postgres:15-alpine
    environment:
      POSTGRES_USER: user
      POSTGRES_PASSWORD: password
      POSTGRES_DB: myapp_db
    volumes:
      - ./init.sql:/docker-entrypoint-initdb.d/init.sql
```

### 7. **Bonusové rozšíření**
- Přidat Redis cache pro zvýšení výkonu
- Vytvořit Dockerfile pro Node.js aplikaci (ne pouze image)
- Přidat healthcheck na obě služby

---

## Cvičení 2: **Nastavení environment proměnných a konfiguračních souborů**

### 1. **Název úkolu**
Správa konfigurací pomocí `.env` souboru a Docker Compose

### 2. **Detailní zadání**
Vytvoř Docker Compose projekt, který používá `.env` soubor pro správu proměnných prostředí.
Aplikace by měla běžet na portu 8080 a přistupovat k databázi pomocí nastavených proměnných.

### 3. **Vstupní data/požadavky**
- Služba: Node.js s Expressem
- Databáze: MySQL (`mysql:8`)
- Použij `.env` soubor pro konfiguraci:
  - `DATABASE_HOST`, `DATABASE_USER`, `DATABASE_PASSWORD`, `DATABASE_NAME`
- Přístup k aplikaci na portu `8080`

### 4. **Očekávaný výstup**
- Soubor `.env` s nastavenými proměnnými
- `docker-compose.yml` používající `.env`
- Aplikace běží a připojuje se k MySQL

### 5. **Hints/Nápověda**
1. V Docker Compose použij `env_file:` pro načtení proměnných z `.env`
2. Použij `environment` v službě jako fallback
3. Ověř, že proměnné jsou správně předány do kontejneru
4. Využij `docker-compose config` pro ověření konfigurace

### 6. **Kostra řešení**
```yaml
version: '3.8'
services:
  web:
    image: node:18-alpine
    env_file:
      - .env
    ports:
      - "8080:8080"
    depends_on:
      - db

  db:
    image: mysql:8
    env_file:
      - .env
```

### 7. **Bonusové rozšíření**
- Vytvoř různé `.env` soubory pro vývoj a produkci (`dev.env`, `prod.env`)
- Přidej použití `docker-compose.override.yml` pro lokální ladění

---

## Cvičení 3: **Sdílení dat mezi kontejnery pomocí volumes**

### 1. **Název úkolu**
Vytvoření sdíleného svazku pro komunikaci mezi službami

### 2. **Detailní zadání**
Vytvoř Docker Compose s třemi službami:
- Frontend (React)
- Backend (Node.js)
- Redis cache
Použij sdílené svazky (volumes) pro přenos dat mezi kontejnery.

### 3. **Vstupní data/požadavky**
- Frontend běží na portu 3000
- Backend na portu 8000
- Redis na výchozím portu 6379
- Použij named volume pro sdílení dat mezi frontend a backendem

### 4. **Očekávaný výstup**
- Docker Compose s třemi službami a sdíleným svazkem
- Data mohou být předávána mezi kontejnery (např. zápis do Redis, čtení frontendem)
- Všechny služby běží spolu

### 5. **Hints/Nápověda**
1. Použij named volumes (`volumes: mydata:/data`)
2. Zajisti správné připojení mezi kontejnery
3. Využij `docker volume ls` pro kontrolu vytvořených svazků
4. Nastav přístupová práva ke sdíleným souborům

### 6. **Kostra řešení**
```yaml
version: '3.8'
services:
  frontend:
    image: node:18-alpine
    volumes:
      - mydata:/app/data
    ports:
      - "3000:3000"

  backend:
    image: node:18-alpine
    volumes:
      - mydata:/app/data
    ports:
      - "8000:8000"

  redis:
    image: redis:alpine

volumes:
  mydata:
```

### 7. **Bonusové rozšíření**
- Implementuj vlastní init skripty pro inicializaci sdílených dat
- Přidej `volume_driver` pro specifickou konfiguraci svazků

---

## Cvičení 4: **Nastavení sítí a připojení mezi službami**

### 1. **Název úkolu**
Vlastní Docker síť pro komunikaci mezi kontejnery

### 2. **Detailní zadání**
Vytvoř Docker Compose konfiguraci s vlastní sítí (`networks`) pro zajištění izolace a správného připojení mezi službami.
Služby:
- Web server (Nginx)
- Backend (Node.js)
-


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

Jistě! Níže najdeš komplexní seznam zdrojů pro kapitolu **Docker Compose**, které zahrnují články, videa, knihy, praktické projekty i online nástroje. Vše je strukturováno v jasném formátu Markdown.

---

## Docker Compose – Doporučené zdroje

### 1. **Články a tutoriály**

1. **[Docker Compose Documentation (Official)](https://docs.docker.com/compose/)**  
   *Popis:* Oficiální dokumentace od Dockeru. Vhodná pro detailní pochopení všech možností a konfigurace `docker-compose.yml`. Zahrnuje příklady, popisy služeb, sítí a svazků.

2. **[Docker Compose Tutorial for Beginners - TechWorld with Nana](https://www.youtube.com/watch?v=Qw9zlEJc6Qg)**  
   *Popis:* Praktický úvodní tutoriál pro začátečníky, který ukazuje, jak vytvářet a spravovat více služeb pomocí `docker-compose.yml`. Vhodné pro rychlé osvojení základů.

3. **[A Complete Guide to Docker Compose](https://www.freecodecamp.org/news/complete-guide-to-docker-compose/)**  
   *Popis:* Komplexní článek, který vysvětluje Docker Compose od základů až po pokročilé funkce. Zahrnuje i řešení běžných problémů při práci s více kontejnery.

4. **[Docker Compose Best Practices](https://docs.docker.com/compose/best-practice/)**  
   *Popis:* Oficiální průvodce osvědčenými postupy pro psaní efektivních a bezpečných `docker-compose.yml` souborů.

5. **[Understanding Docker Compose File Structure](https://www.digitalocean.com/community/tutorials/how-to-use-docker-compose-to-run-multiple-containers)**  
   *Popis:* Článek od DigitalOcean, který se zaměřuje na strukturu `docker-compose.yml` a ukazuje konkrétní příklady použití.

---

### 2. **YouTube videa**

1. **[Docker Compose Tutorial for Beginners - FreeCodeCamp](https://www.youtube.com/watch?v=Qw9zlEJc6Qg)**  
   *Délka:* 27 minut  
   *Popis:* Úvodní video pro začátečníky, které ukazuje, jak nastavit a spustit více služeb pomocí Docker Compose.

2. **[Docker Compose Crash Course - The Net Ninja](https://www.youtube.com/watch?v=V306bN97358)**  
   *Délka:* 32 minut  
   *Popis:* Zábavný a stručný crash course, který vysvětluje Docker Compose podrobněji, s příklady z reálného světa.

3. **[How to Use Docker Compose with PostgreSQL and Node.js](https://www.youtube.com/watch?v=4127X0xJQd0)**  
   *Délka:* 45 minut  
   *Popis:* Praktický návod, jak propojit PostgreSQL a Node.js pomocí Docker Compose – skvělé pro začátečníky.

---

### 3. **Knihy a oficiální dokumentace**

1. **[Docker in Action, Second Edition](https://www.manning.com/books/docker-in-action-second-edition)**  
   *Popis:* Rozsáhlá kniha o Dockeru včetně kapitoly o Docker Compose. Skvělé pro čtení a detailní pochopení systému.

2. **[Docker Official Documentation on Compose](https://docs.docker.com/compose/overview/)**  
   *Popis:* Kompletní a oficiální dokumentace, která je základem každého vývojá


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ docker-compose.yml structure</li>
<li>✓ Services definition</li>
<li>✓ Multi-container applications</li>
<li>✓ Networks a volumes in compose</li>
<li>✓ Environment variables</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 23 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:45:16*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

